In [1]:
from src.inference import evaluate_model

In [2]:

def interpret(model_path):
    cer_val, wer_val = evaluate_model(
    model_path=model_path, 
    datapath= model_path + '/val.csv' 
    )
    cer_test, wer_test = evaluate_model(
    model_path=model_path,
    datapath='data/testset.npy'
    )
    print(f"Validation CER: {cer_val}, Validation WER: {wer_val}")
    print(f"Test CER: {cer_test}, Test WER: {wer_test}")
    
    


In [3]:
interpret('results/htr_net_5k_data')


100%|██████████| 247/247 [00:02<00:00, 90.75it/s]

Validation CER: 0.1013065937405044, Validation WER: 0.21330902461257975
Test CER: 0.20850202429149794, Test WER: 0.3967611336032389


In [4]:
interpret('results/htr_net_no_custom_new_split')

100%|██████████| 247/247 [00:02<00:00, 90.75it/s]

Validation CER: 0.15200000000000002, Validation WER: 0.29
Test CER: 0.3600539811066127, Test WER: 0.6072874493927125


In [5]:
interpret('results/cnn_bilstm_entire_dataset')


100%|██████████| 247/247 [00:01<00:00, 185.20it/s]

Validation CER: 0.10475513428120062, Validation WER: 0.22843601895734597
Test CER: 0.2012145748987854, Test WER: 0.3724696356275304
